In [1]:
import numpy as np
np.random.seed(42)
class Koikoi:
    def __init__(self,num_player_hand=8,num_field=8,num_battle=12,
                 gokou=10,yonkou=8,ame=7,sankou=5,hanami=5,tukimi=5,inosika=5,akatan=5,
                 aotan=5,tane=1,tan=1,kasu=1):
        self.num_player_hand=num_player_hand
        self.num_field=num_field
        self.num_battle=num_battle
        self.yaku = {'gokou': gokou,'yonkou': yonkou,'ame': ame,'sankou': sankou,
                     'hanami': hanami,'tukimi': tukimi,'inosika': inosika,'akatan': akatan,
                     'aotan': aotan,'tane': tane,'tan': tan,'kasu': kasu}
        self.player_total_score=0
        self.opponent_total_score=0
        self.state=None
        self.player_hand=[]
        self.opponent_hand=[]
        self.player_deck=[]
        self.opponent_deck=[]
        self.field=[]
        self.card_index=[]
        self.game_deck=[]
        self.deck_open_count=0
        self.deck=['1_1','1_1','1_5','1_20','2_1','2_1','2_5','2_X',
                    '3_1','3_1','3_5','3_20','4_1','4_1','4_5','4_X',
                    '5_1','5_1','5_5','5_X','6_1','6_1','6_5','6_X',
                    '7_1','7_1','7_5','7_X','8_1','8_1','8_5','8_20',
                    '9_1','9_1','9_5','9_X','x_1','x_1','x_5','x_X',
                    'y_1','y_5','y_10','y_20','z_1','z_1','z_1','z_20']
        self.card_label={'1_1':1,'1_5':2,'1_20':3,'2_1':4,'2_5':5,'2_X':6,
                         '3_1':7,'3_5':8,'3_20':9,'4_1':10,'4_5':11,'4_X':12,
                         '5_1':13,'5_5':14,'5_X':15,'6_1':16,'6_5':17,'6_X':18,
                         '7_1':19,'7_5':20,'7_X':21,'8_1':22,'8_5':23,'8_20':24,
                         '9_1':25,'9_5':26,'9_X':27,'x_1':28,'x_5':29,'x_X':30,
                         'y_1':31,'y_5':32,'y_10':33,'y_20':34,'z_1':35,'z_20':36}
        self.max_num_field=18
        self.max_num_player_deck=32
        self.hand_action_space=[i for i in range(8*(self.max_num_field-1))]
        self.deck_action_space=[i for i in range(self.max_num_field-1)]
        self.player_score=0
        self.opponent_score=0
        self.player_reward=0
        self.opponent_reward=0
        self.koikoi_player=False
        self.koikoi_opponent=False
        self.player_score_action=False
        self.opponent_score_action=False
        self.fin=False

    def shuffle(self,p_hand=8,field=8):
        self.card_index=np.random.choice(48, size=p_hand*4+field, replace=False)
        for i in range(p_hand*4+field):
            if i<=p_hand-1:
                self.player_hand.append(self.deck[self.card_index[i]])
            elif i<=2*p_hand-1:
                self.opponent_hand.append(self.deck[self.card_index[i]])
            elif i<=3*p_hand-1:
                self.field.append(self.deck[self.card_index[i]])
            else:
                self.game_deck.append(self.deck[self.card_index[i]])

    #state=input data for machine learning
    def calculate_state(self,actor,deck_card=None):
        state = [0] * 97
        #former or latter
        if actor == 0:
            state[0] = 0 #former

        #encoding player hand
            for i in range(self.num_player_hand):
                if i < len(self.player_hand):
                    card = self.player_hand[i]
                    card_index = self.card_label[card]
                    state[i+1] = card_index

        #encoding field hand
            for i in range(self.max_num_field):#self.max_num_field=18 in logical
                if i < len(self.field):
                    card = self.field[i]
                    card_index = self.card_label[card]
                    state[i+1+self.num_player_hand] = card_index

        #encoding player deck card
            for i in range(self.max_num_player_deck):
                if i < len(self.player_deck):
                    card = self.player_deck[i]
                    card_index = self.card_label[card]
                    state[i+1+self.num_player_hand+self.max_num_field] = card_index

        #encoding opponent deck card
            for i in range(self.max_num_player_deck):
                if i < len(self.opponent_deck):
                    card = self.opponent_deck[i]
                    card_index = self.card_label[card]
                    state[i+1+self.num_player_hand+self.max_num_field+self.max_num_player_deck] = card_index

        #total score
            state[1+self.num_player_hand+self.max_num_field+self.max_num_player_deck*2]=self.player_total_score
            state[2+self.num_player_hand+self.max_num_field+self.max_num_player_deck*2]=self.opponent_total_score
            if deck_card!=None:
                card_index = self.card_label[card]
                state[-3] = card_index
            if self.koikoi_player:
                state[-2]=1
            if self.koikoi_opponent:
                state[-1]=1

        #latter player
        else:
            state[0] = 1 #latter

        #encoding player hand
            for i in range(self.num_player_hand):
                if i < len(self.opponent_hand):
                    card = self.opponent_hand[i]
                    card_index = self.card_label[card]
                    state[i+1] = card_index

        #encoding field hand
            for i in range(self.max_num_field):#self.max_num_field=18 in logical
                if i < len(self.field):
                    card = self.field[i]
                    card_index = self.card_label[card]
                    state[i+1+self.num_player_hand] = card_index

        #encoding player deck card
            for i in range(self.max_num_player_deck):
                if i < len(self.opponent_deck):
                    card = self.opponent_deck[i]
                    card_index = self.card_label[card]
                    state[i+1+self.num_player_hand+self.max_num_field] = card_index

        #encoding opponent deck card
            for i in range(self.max_num_player_deck):
                if i < len(self.player_deck):
                    card = self.player_deck[i]
                    card_index = self.card_label[card]
                    state[i+1+self.num_player_hand+self.max_num_field+self.max_num_player_deck] = card_index

        #total score
            state[1+self.num_player_hand+self.max_num_field+self.max_num_player_deck*2]=self.opponent_total_score
            state[2+self.num_player_hand+self.max_num_field+self.max_num_player_deck*2]=self.player_total_score
            if deck_card!=None:
                card_index = self.card_label[card]
                state[-3] = card_index
            if self.koikoi_opponent:
                state[-2]=1
            if self.koikoi_player:
                state[-1]=1
        return state

    def count_total_score(self,score,actor):
        if actor==0:
            self.player_total_score+=score
        else:
            self.opponent_total_score+=score


    def hand_action(self,action,actor):
        #former player
        if actor==0:
            #selection of hand card
            if action//self.max_num_field>=len(self.player_hand):
                hand_card=self.player_hand[-1]
                if action%self.max_num_field>=len(self.field):
                    self.field.append(hand_card)
                    del self.player_hand[-1]
                else:
                    field_card=self.field[action%self.max_num_field]
                    #confirmation of match between hand and field cards
                    if hand_card[0]==field_card[0]:
                        self.player_deck.append(hand_card)
                        self.player_deck.append(field_card)
                        del self.player_hand[-1]
                        del self.field[action%self.max_num_field]
                    else:
                        self.field.append(hand_card)
                        del self.player_hand[-1]
            else:
                hand_card=self.player_hand[action//self.max_num_field]
                #selection of field card
                if action%self.max_num_field>=len(self.field):
                    self.field.append(hand_card)
                    del self.player_hand[action//self.max_num_field]
                else:
                    field_card=self.field[action%self.max_num_field]
                    #confirmation of match between hand and field cards
                    if hand_card[0]==field_card[0]:
                        self.player_deck.append(hand_card)
                        self.player_deck.append(field_card)
                        del self.player_hand[action//self.max_num_field]
                        del self.field[action%self.max_num_field]
                    else:
                        self.field.append(hand_card)
                        del self.player_hand[action//self.max_num_field]

        #latter player
        if actor==1:
            #selection of hand card
            if action//self.max_num_field>=len(self.opponent_hand):
                hand_card=self.opponent_hand[-1]
                if action%self.max_num_field>=len(self.field):
                    self.field.append(hand_card)
                    del self.opponent_hand[-1]
                else:
                    field_card=self.field[action%self.max_num_field]
                    #confirmation of match between hand and field cards
                    if hand_card[0]==field_card[0]:
                        self.opponent_deck.append(hand_card)
                        self.opponent_deck.append(field_card)
                        del self.opponent_hand[-1]
                        del self.field[action%self.max_num_field]
                    else:
                        self.field.append(hand_card)
                        del self.opponent_hand[-1]
            else:
                hand_card=self.opponent_hand[action//self.max_num_field]
                #selection of field card
                if action%self.max_num_field>=len(self.field):
                    self.field.append(hand_card)
                    del self.opponent_hand[action//self.max_num_field]
                else:
                    field_card=self.field[action%self.max_num_field]
                    #confirmation of match between hand and field cards
                    if hand_card[0]==field_card[0]:
                        self.opponent_deck.append(hand_card)
                        self.opponent_deck.append(field_card)
                        del self.opponent_hand[action//self.max_num_field]
                        del self.field[action%self.max_num_field]
                    else:
                        self.field.append(hand_card)
                        del self.opponent_hand[action//self.max_num_field]

    def deck_card_open(self):
        card=self.game_deck[self.deck_open_count]
        self.deck_open_count+=1
        return card

    def deck_action(self,action,deck_card,actor):
        #former player
        if actor==0:
            #selection of field card
            if action>=len(self.field):
                self.field.append(deck_card)
            else:
                field_card=self.field[action]
                #confirmation of match between deck and field cards
                if deck_card[0]==field_card[0]:
                    self.player_deck.append(deck_card)
                    self.player_deck.append(field_card)
                    del self.field[action]
                else:
                    self.field.append(deck_card)
        #latter player
        else:
            #selection of field card
            if action>=len(self.field):
                self.field.append(deck_card)
            else:
                field_card=self.field[action]
                #confirmation of match between deck and field cards
                if deck_card[0]==field_card[0]:
                    self.opponent_deck.append(deck_card)
                    self.opponent_deck.append(field_card)
                    del self.field[action]
                else:
                    self.field.append(deck_card)

    def koikoi_action(self,action,actor):
        #former player
        if actor==0:
            #koikoi
            if action==0:
                self.koikoi_player=True
                self.player_score_action=False
                pass
            else:
                self.fin=True
                score=self.score_checker(actor)
                self.count_total_score(score,actor)
        #latter player
        else:
            #koikoi
            if action==0:
                self.koikoi_opponent=True
                self.opponent_score_action=False
                pass
            else:
                self.fin=True
                score=self.score_checker(actor)
                self.count_total_score(score,actor)

    def calculate_score(self,actor):
        score_1=0
        score_5=0
        score_10=0
        score_20=0
        ame=0
        score=0
        if actor==0:
            deck_score=self.player_deck
            for card in deck_score:
                if card[2]=='1':
                    score_1+=1
                elif card[2]=='5':
                    score_5+=1
                elif card[2]=='X':
                    score_10+=1
                elif card[2]=='2' and card!='y_20':
                    score_20+=1
                elif card=='y_20':
                    ame+=1

            if score_1>=10:
                a=score_1-10
                score+=a+self.yaku['kasu']
            if score_5>=5:
                a=score_5-5
                score+=a+self.yaku['tan']
            if score_10>=5:
                a=score_10-5
                score+=a+self.yaku['tane']
            inosika_strings = ["x_X", "7_X","6_X"]
            if all(string in deck_score for string in inosika_strings):
                score+=self.yaku['inosika']
            aotan_strings = ["6_5", "9_5","x_5"]
            if all(string in deck_score for string in aotan_strings):
                score+=self.yaku['aotan']
            akatan_strings = ["1_5", "2_5","3_5"]
            if all(string in deck_score for string in aotan_strings):
                score+=self.yaku['akatan']
            hanami_strings = ["3_20", "9_X"]
            if all(string in deck_score for string in aotan_strings):
                score+=self.yaku['hanami']
            tukimi_strings = ["8_20", "9_X"]
            if all(string in deck_score for string in aotan_strings):
                score+=self.yaku['tukimi']
            if score_20==3 and ame==0:
                score+=self.yaku['sankou']
            if score_20==4 and ame==0:
                score+=self.yaku['yonkou']
            if score_20==3 and ame==1:
                score+=self.yaku['ame']
            if score_20==4 and ame==1:
                score+=self.yaku['gokou']

            if self.koikoi_opponent==True:
                score=score*2
                self.player_score=score
            else:
                self.player_score=score
        else:
            deck_score=self.opponent_deck
            for card in deck_score:
                if card[2]=='1':
                    score_1+=1
                elif card[2]=='5':
                    score_5+=1
                elif card[2]=='X':
                    score_10+=1
                elif card[2]=='2' and card!='y_20':
                    score_20+=1
                elif card=='y_20':
                    ame+=1

            if score_1>=10:
                a=score_1-10
                score+=a+self.yaku['kasu']
            if score_5>=5:
                a=score_5-5
                score+=a+self.yaku['tan']
            if score_10>=5:
                a=score_10-5
                score+=a+self.yaku['tane']
            inosika_strings = ["x_X", "7_X","6_X"]
            if all(string in deck_score for string in inosika_strings):
                score+=self.yaku['inosika']
            aotan_strings = ["6_5", "9_5","x_5"]
            if all(string in deck_score for string in aotan_strings):
                score+=self.yaku['aotan']
            akatan_strings = ["1_5","2_5","3_5"]
            if all(string in deck_score for string in akatan_strings):
                score+=self.yaku['akatan']
            hanami_strings = ["3_20", "9_X"]
            if all(string in deck_score for string in hanami_strings):
                score+=self.yaku['hanami']
            tukimi_strings = ["8_20", "9_X"]
            if all(string in deck_score for string in tukimi_strings):
                score+=self.yaku['tukimi']
            if score_20==3 and ame==0:
                score+=self.yaku['sankou']
            if score_20==4 and ame==0:
                score+=self.yaku['yonkou']
            if score_20==3 and ame==1:
                score+=self.yaku['ame']
            if score_20==4 and ame==1:
                score+=self.yaku['gokou']

            if self.koikoi_player==True:
                score=score*2
                self.opponent_score=score
            else:
                self.opponent_score=score

        return score

    def score_checker(self, actor):
        if actor == 0:
            prev_score = self.player_score
            current_score = self.calculate_score(actor)
            if prev_score != current_score:
                self.player_score_action = True
            else:
                pass
        else:
            prev_score = self.opponent_score
            current_score = self.calculate_score(actor)
            if prev_score != current_score:
                self.opponent_score_action = True
            else:
                pass

        return current_score

    def easy_reward(self,actor):
        if actor == 0:
            prev_score = self.player_score
            current_score = self.score_checker(actor)
            reward=current_score-prev_score+len(self.player_deck)
        else:
            prev_score = self.opponent_score
            current_score = self.score_checker(actor)
            reward=current_score-prev_score+len(self.opponent_deck)
        return reward

    def koikoi_reward(self,actor):
        if actor == 0:
            reward=self.player_total_score
        else:
            reward=self.opponent_total_score

        return reward

    def reward(self,actor):
        if actor == 0:
            prev_score = self.player_score
            current_score = self.score_checker(actor)
            reward=current_score-prev_score
        else:
            prev_score = self.opponent_score
            current_score = self.score_checker(actor)
            reward=current_score-prev_score
        return reward

    def hand_action_step(self,action,actor):
        self.hand_action(action,actor)

    def deck_action_step(self,action,deck_card,actor):
        self.deck_action(action,deck_card,actor)

    def koikoi_action_step(self,action,actor):
        self.koikoi_action(action,actor)

    def game_reset(self,):#1 game fin
        self.player_hand=[]
        self.opponent_hand=[]
        self.player_deck=[]
        self.opponent_deck=[]
        self.field=[]
        self.card_index=[]
        self.game_deck=[]
        self.deck_open_count=0
        self.player_score=0
        self.opponent_score=0
        self.player_score_action=False
        self.opponent_score_action=False
        self.koikoi_player=False
        self.koikoi_opponent=False
        self.fin=False
        pass

    def reset(self):#reset total score
        self.player_total_score=0
        self.opponent_total_score=0
        pass


In [2]:
import copy
from collections import deque
import random
import numpy as np
from sklearn import metrics
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_

class ReplayBuffer:
    def __init__(self, buffer_size, batch_size):
        self.buffer = deque(maxlen=buffer_size)
        self.batch_size = batch_size

    def add(self, state, action, reward, next_state):
        data = (state, action, reward, next_state)
        self.buffer.append(data)

    def __len__(self):
        return len(self.buffer)

    def get_batch(self):
        data = random.sample(self.buffer, self.batch_size)

        state = torch.tensor(np.stack([x[0] for x in data]).astype(np.float32))
        action = torch.tensor(np.array([x[1] for x in data]).astype(np.float32))
        reward = torch.tensor(np.array([x[2] for x in data]).astype(np.float32))
        next_state = torch.tensor(np.stack([x[3] for x in data]).astype(np.float32))
        return state, action, reward, next_state


class hand_QNet(nn.Module):
    def __init__(self,action_size):
        super().__init__()
        self.fc1 = nn.Linear(97, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        h = F.relu(h)
        h = self.fc3(h)
        return h

class deck_QNet(nn.Module):
    def __init__(self, action_size):
        super().__init__()
        self.l1 = nn.Linear(97, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x

class koikoi_QNet(nn.Module):
    def __init__(self, action_size):
        super().__init__()
        self.l1 = nn.Linear(97, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        return x


In [3]:
class DQNAgent:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.gamma = 0.98
        self.lr = 0.005
        self.epsilon = 0.1
        self.koikoi_epsilon = 0.2
        self.buffer_size = 1000
        self.batch_size = 128
        self.num_hand=8
        self.hand_action_size = self.num_hand*(18-1)
        self.deck_action_size = 18
        self.koikoi_action_size = 2
        self.hand_replay_buffer = ReplayBuffer(self.buffer_size, self.batch_size)
        self.deck_replay_buffer = ReplayBuffer(self.buffer_size, self.batch_size)
        self.koikoi_replay_buffer = ReplayBuffer(self.buffer_size, self.batch_size)
        self.hand_qnet = hand_QNet(self.hand_action_size).to(self.device)
        self.hand_qnet_target = hand_QNet(self.hand_action_size).to(self.device)
        self.deck_qnet = deck_QNet(self.deck_action_size).to(self.device)
        self.deck_qnet_target = deck_QNet(self.deck_action_size).to(self.device)
        self.koikoi_qnet = koikoi_QNet(self.koikoi_action_size).to(self.device)
        self.koikoi_qnet_target = koikoi_QNet(self.koikoi_action_size).to(self.device)
        self.hand_optimizer = optim.Adam(self.hand_qnet.parameters(), lr=self.lr)
        self.deck_optimizer = optim.Adam(self.deck_qnet.parameters(), lr=self.lr)
        self.koikoi_optimizer = optim.Adam(self.koikoi_qnet.parameters(), lr=self.lr)

    def get_hand_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.hand_action_size)
        else:
            state = torch.tensor(state, dtype=torch.float32,device=self.device)
            qs = self.hand_qnet(state)
            return qs.argmax().item()

    def get_deck_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.deck_action_size)
        else:
            state = torch.tensor(state, dtype=torch.float32,device=self.device)
            qs = self.deck_qnet(state)
            return qs.argmax().item()

    def get_koikoi_action(self, state):
        if np.random.rand() < self.koikoi_epsilon:
            return 1 #return game finish in epsiron
        else:
            state = torch.tensor(state, dtype=torch.float32,device=self.device)
            qs = self.koikoi_qnet(state)
            return qs.argmax().item()


    def hand_action_update(self, state, action, reward, next_state):
        self.hand_replay_buffer.add(state, action, reward, next_state)
        if len(self.hand_replay_buffer) < self.batch_size:
            return

        state, action, reward, next_state= self.hand_replay_buffer.get_batch()
        state, action, reward, next_state = state.to(self.device), action.to(self.device), reward.to(self.device), next_state.to(self.device)
        qs = self.hand_qnet(state)
        q = qs[torch.arange(len(action)), action.long()]

        next_qs = self.hand_qnet_target(next_state)
        next_q = next_qs.max(1)[0]

        next_q.detach()
        target = reward +  self.gamma * next_q

        loss_fn = nn.MSELoss()
        loss = loss_fn(q, target)

        self.hand_optimizer.zero_grad()
        loss.backward()
        self.hand_optimizer.step()

    def deck_action_update(self, state, action, reward, next_state):
        self.deck_replay_buffer.add(state, action, reward, next_state)
        if len(self.deck_replay_buffer) < self.batch_size:
            return

        state, action, reward, next_state= self.deck_replay_buffer.get_batch()
        state, action, reward, next_state = state.to(self.device), action.to(self.device), reward.to(self.device), next_state.to(self.device)
        qs = self.deck_qnet(state)
        q = qs[torch.arange(self.batch_size), action.long()]

        next_qs = self.deck_qnet_target(next_state)
        next_q = next_qs.max(1)[0]

        next_q.detach()
        target = reward +  self.gamma * next_q

        loss_fn = nn.MSELoss()
        loss = loss_fn(q, target)

        self.deck_optimizer.zero_grad()
        loss.backward()
        self.deck_optimizer.step()

    def koikoi_action_update(self, state, action, reward, next_state):
        self.koikoi_replay_buffer.add(state, action, reward, next_state)
        if len(self.koikoi_replay_buffer) < self.batch_size:
            return

        state, action, reward, next_state= self.koikoi_replay_buffer.get_batch()
        state, action, reward, next_state = state.to(self.device), action.to(self.device), reward.to(self.device), next_state.to(self.device)
        qs = self.koikoi_qnet(state)
        q = qs[torch.arange(len(action)), action.long()]

        next_qs = self.koikoi_qnet_target(next_state)
        next_q = next_qs.max(1)[0]

        next_q.detach()
        target = reward +  self.gamma * next_q

        loss_fn = nn.MSELoss()
        loss = loss_fn(q, target)

        self.koikoi_optimizer.zero_grad()
        loss.backward()
        self.koikoi_optimizer.step()

    def sync_qnet(self,qnet):
        if qnet=='hand':
            self.hand_qnet_target.load_state_dict(self.hand_qnet.state_dict())
        elif qnet=='deck':
            self.deck_qnet_target.load_state_dict(self.deck_qnet.state_dict())
        else:
            self.koikoi_qnet_target.load_state_dict(self.koikoi_qnet.state_dict())

In [4]:
def play_turn(agent,env,actor):
    #hand action
    state = tuple(env.calculate_state(actor=actor))
    hand_action = agent.get_hand_action(state)
    env.hand_action_step(hand_action, actor=actor)
    reward_hand_action = env.easy_reward(actor=actor)
    next_state = tuple(env.calculate_state(actor=actor))
    agent.hand_action_update(state, hand_action, reward_hand_action, next_state)

    #deck action
    deck_card = env.deck_card_open()
    state = tuple(env.calculate_state(actor=actor, deck_card=deck_card))
    deck_action = agent.get_deck_action(state)
    env.deck_action_step(deck_action, deck_card, actor=actor)
    reward_deck_action = env.easy_reward(actor=actor)
    next_state = tuple(env.calculate_state(actor=actor))
    agent.deck_action_update(state, deck_action, reward_deck_action, next_state)

    #koikoi action
    score = env.score_checker(actor=actor)
    if env.player_score_action:
        print('Player koikoi challenge:',actor)
        print('Player deck:', env.player_deck)
        print('Player score:', env.player_score)
        state = tuple(env.calculate_state(actor=actor))
        koikoi_action = agent.get_koikoi_action(state)
        env.koikoi_action_step(koikoi_action, actor=actor)
        reward_koikoi_action = env.koikoi_reward(actor=actor)
        if env.fin:
            next_state = tuple(env.calculate_state(actor=actor))
            agent.koikoi_action_update(state, hand_action, reward_koikoi_action, next_state)
            env.game_reset()
            print('Game finished')
            return True
        else:
            next_state = tuple(env.calculate_state(actor=actor))
            agent.koikoi_action_update(state, hand_action, reward_koikoi_action, next_state)
            print('Koikoi')
    return False

In [5]:
sync_interval = 20
env = Koikoi()
agent1 = DQNAgent()
agent2 = DQNAgent()
reward_history = []
episodes = 1000
for i in range(1,11):    
    for episode in range(episodes):
        if episode%50==0:
            print('episode:',episode)
        env.reset()
        for game in range(env.num_battle):#12 battles
            action_num = 0
            env.game_reset()
            env.shuffle()
            if random.random() < 0.5:
                former = agent1
                latter = agent2
            else:
                former = agent2
                latter = agent1
            while action_num < env.num_player_hand:
                # former
                if play_turn(former, env, actor=0):
                    break
                # latter
                if play_turn(latter, env, actor=1):
                    break
                #count action(The maximum number of counts is until all hand cards are depleted.)
                action_num += 1
            #draw game
            env.game_reset()
    
        #12battle終わったら勝った方にrewardをh倍負けた方に1/h倍
        if env.player_total_score>env.opponent_total_score:
            print('-----------------------------------------------')
            print('player win')
            print('total player score:',env.player_total_score)
            print('total opponent score:',env.opponent_total_score)
            print('-----------------------------------------------')
    
        elif env.player_total_score==env.opponent_total_score:
            #storage reward data to buffer
            pass
    
        else:
            print('-----------------------------------------------')
            print('opponent win')
            print('total opponent score:',env.opponent_total_score)
            print('total player score:',env.player_total_score)
            print('-----------------------------------------------')
        env.reset()
        if episode % sync_interval == 0:
            for qnet in ['hand','deck','koikoi']:
                agent1.sync_qnet(qnet)
                agent2.sync_qnet(qnet)
                
    ep=1000*i
    #storage model
    hand_model_name = f"C:/Users/Machi/Documents/koikoi/DQN1/models/hand_model_{ep}.pth"
    deck_model_name = f"C:/Users/Machi/Documents/koikoi/DQN1/models/deck_model_{ep}.pth"
    koikoi_model_name = f"C:/Users/Machi/Documents/koikoi/DQN1/models/koikoi_model_{ep}.pth"
    model_names=[hand_model_name,deck_model_name,koikoi_model_name]
    models=[agent1.hand_qnet,agent1.deck_qnet,agent1.koikoi_qnet]
    for name, model in zip(model_names, models):
        torch.save(model.state_dict(), name)

episode: 0
Player koikoi challenge: 0
Player deck: ['6_X', '6_1', '7_1', '7_X', 'x_X', 'x_5']
Player score: 5
Game finished
-----------------------------------------------
player win
total player score: 5
total opponent score: 0
-----------------------------------------------
episode: 50
episode: 100
episode: 150
episode: 200
episode: 250
Player koikoi challenge: 0
Player deck: ['6_X', '6_1', '7_5', '7_X', 'x_1', 'x_X']
Player score: 5
Game finished
-----------------------------------------------
player win
total player score: 5
total opponent score: 0
-----------------------------------------------
episode: 300
episode: 350
episode: 400
Player koikoi challenge: 0
Player deck: ['9_X', '9_1', 'z_1', 'z_20', '3_1', '3_20', '1_20', '1_1']
Player score: 5
Game finished
-----------------------------------------------
player win
total player score: 5
total opponent score: 0
-----------------------------------------------
episode: 450
episode: 500
episode: 550
episode: 600
episode: 650
episod